In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from scipy.sparse import hstack
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
url = "https://raw.githubusercontent.com/AREEG94FAHAD/TaskComplexityEval-24/main/problems_data.jsonl"

df = pd.read_json(url, lines=True)

In [ ]:
df['text'] = df['description']+' '+ df['input_description']+' '+df['output_description']

In [ ]:
print(df['text'][0])

Unununium (Uuu) was the name of the chemical
    element with atom number 111, until it changed to
    Röntgenium (Rg) in 2004. These heavy elements are very
    unstable and have only been synthesized in a few
    laboratories.
You have just been hired by one of these labs to optimize
    the algorithms used in simulations. For example, when
    simulating complicated chemical reactions, it is important to
    keep track of how many particles there are, and this is done by
    counting connected components in a graph.
Currently, the lab has some Python code (see attachments)
    that takes an undirected graph and outputs the number of
    connected components. As you can see, this code is based on
    everyone’s favourite data structure union-find1.
After looking at the code for a while, you notice that it
    actually has a bug in it! The code still gives correct answers,
    but the bug could cause it to run inefficiently. Your task is
    to construct a graph with a given number of

In [ ]:
df = df[~(df["text"].str.strip() == "")]

In [ ]:
df['text'] = df['text'].str.replace('$', '', regex=False)
df['text']=df['text'].str.lower()

In [ ]:
def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 3. Remove extra whitespace
    text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. Tokenization (split into words)
    tokens = nltk.word_tokenize(text)

    # 5. Remove Stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 6. Lemmatization (reduce words to their base form)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # 7. Rejoin tokens into a clean string (optional, depending on next step)
    cleaned_text = ' '.join(lemmatized_tokens)

    return cleaned_text

In [ ]:
df['text2']=df['text'].apply(preprocess_text)

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 3),
    min_df=3,
    max_df=0.85,
    sublinear_tf=True
)

X_tfidf = tfidf.fit_transform(df['text2'])
#tf-idf feature

In [ ]:
y1 = df['problem_class']
y2 = df['problem_score']

In [ ]:
def text_structure(text):
    words = text.split()

    return pd.Series({
        "log_num_words": np.log1p(len(words)),
        #"num_sentences": text.count('.'),
        #"avg_word_length": np.mean([len(w) for w in words]) if words else 0.0,
        #"num_lines": text.count('\n'),
        "log_num_lines":np.log1p(text.count('\n'))
    })

In [ ]:
try2 = df["text"].apply(text_structure)
try2_dash = hstack([X_tfidf , try2])

In [ ]:
KEYWORD_BUCKETS = {
    "spatial": [
        "grid", "matrix", "cell", "square",
        "row", "column",
        "coordinate", "position", "location",
        "distance", "path", "move", "step",
        "neighbor", "adjacent", "direction",
        "north", "south", "left", "right",
        "x", "y"
    ]}

In [ ]:
def keyword_bucket_features(text):
    if not isinstance(text, str) or text.strip() == "":
        return pd.Series({
            f"{name}_count": 0
            for name in KEYWORD_BUCKETS
        } | {
            f"{name}_flag": 0
            for name in KEYWORD_BUCKETS
        })

    text = text.lower()

    features = {}

    for bucket_name, words in KEYWORD_BUCKETS.items():
        count = 0
        for w in words:
            if w in text:
                count += text.count(w)

        features[f"{bucket_name}_count"] = count
        features[f"{bucket_name}_flag"] = int(count > 0)

    return pd.Series(features)

In [ ]:
X_bucket = df["text"].apply(keyword_bucket_features)

In [ ]:
from scipy.sparse import csr_matrix
try3 = X_bucket[['spatial_count']]
try3 = try3.fillna(0)
#try3_scaled = scaler.fit_transform(try3)
try3_sparse = csr_matrix(try3)
X3_dash = hstack([try3_sparse , try2_dash])

In [ ]:
def fet2(text):
    if not isinstance(text, str):
        return 0
    nums = [int(n) for n in re.findall(r"\d+", text)]
    return max(nums) if nums else 0

In [ ]:
try5 = df["text"].apply(fet2)
try5_np = try5.to_numpy(dtype=np.float64)

try5_log = np.log1p(try5_np)

In [ ]:
X5 = hstack([X3_dash , try5_log.reshape(-1, 1)])

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

clf.fit(X5, y1)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
CLASS_RANGES = {
    "easy": (1.1 , 2.8),
    "medium": (2.8 , 5.5),
    "hard": (5.5 , 9.7),
}

In [ ]:
for cls, (lo, hi) in CLASS_RANGES.items():
    mask = df["problem_class"] == cls
    df.loc[mask, "y_rel"] = (df.loc[mask, "problem_score"] - lo) / (hi - lo)

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X5, df["y_rel"])

Ridge()

In [ ]:
import joblib

joblib.dump(tfidf, "tfidf.pkl")
joblib.dump(clf, "classifier.pkl")
joblib.dump(ridge, "regressor.pkl")


['regressor.pkl']